## Fine-Tuning Neural Network Hyperparameters

The flexibility of neural networks is also one of their drawbacks: there are many hyperparameters to tweak. Not only you can use any imaginable network architecture, but even in a simple MLP you can change the number of layers, the number of neurons per layer, the type of activation function to use in each layer, the weight initialization logic, and much more. How do you know what combination of hyperparameters is the best for your task?

One option is to simply try many combinations of hyperparameters and see which one works best on the validation set (or use K-fold cross-validation). For example, we can use GridSearchCV to explore the hyperparameter space, as we did in Chapter2. To do this, we need to wrap our Keras models in objects that mimic regular Scikit-Learn regressors. The first step is to create a function that will build an compile a Keras model, given a set of parameters:

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

        This function creates a simple Sequential model for univariate regression (only one output neuron), with 
    the given input shape and the given number of hidden layers and neurons, and compiles it using SGD optimizer
    configired with the specified learning rate. It is good practice to provide reasonable defaults to as many
    hyperparameters as you can, as Scikit-Learn does.

Next let's create a KerasRegressor based on this build_model() function:

In [4]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

The KerasRegressor object is a thin wrapper around the Keras model built using build_model(). Since we did not specify any hyperparameters when creating it, it will use the default hyperparameters we defined in build_model(). Now we can use this object like a regular Scikit-Learn regressor: we can train it using its fit() method, then evaluate it using its score() method, and use it to make predictions using its predict() method, as you can see in the following code:

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [6]:
#loading the data
housing = fetch_california_housing()
#splitting into train, validation and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)
#scale all the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [7]:
keras_reg.fit(X_train, y_train, epochs=100,
             validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)],
             verbose=0,)
mse_test = keras_reg.score(X_test, y_test)
X_new = X_test[:5]
y_pred = keras_reg.predict(X_new)

5160/5160 [==============================] - 0s 31us/sample - loss: 0.3485


Note that any extra parameter you pass to the fit() method will get passed to the underlying Keras mdel. Also note that score will be opposite of the MSE because Scikit-Learn wants scores, not losses(i.e., higher should be better)

We don't want to train and evaluate a single model like this, though we want to train hundreds of variants and see which one performs best on the validation set. Since there are many hyperparameters, it is preferable to use a randomized search rather than grid search (as we discussed in Chapter 2). Let's try to explore the number of hidden layers, the number of neurons, and the learning rate:

In [10]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [11]:
import numpy as np

In [14]:
param_distribs = {
    'n_hidden': [0,1,2,3],
    'n_neurons': np.arange(1,100),
    'learning_rate': reciprocal(3e-4, 3e-2),
}


rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 89us/sample - loss: 3.9977 - val_loss: 2.2976
Epoch 2/100
7740/7740 [==============================] - 0s 62us/sample - loss: 1.6552 - val_loss: 1.2166
Epoch 3/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.9782 - val_loss: 0.8542
Epoch 4/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.7525 - val_loss: 0.7266
Epoch 5/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.6705 - val_loss: 0.6762
Epoch 6/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.6370 - val_loss: 0.6527
Epoch 7/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.6200 - val_loss: 0.6387
Epoch 8/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.6094 - val_loss: 0.6284
Epoch 9/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.6013 - val_loss: 0.619

7740/7740 [==============================] - 0s 58us/sample - loss: 0.5232 - val_loss: 0.5269
Epoch 78/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5230 - val_loss: 0.5268
Epoch 79/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5230 - val_loss: 0.5269
Epoch 80/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5230 - val_loss: 0.5268
Epoch 81/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5230 - val_loss: 0.5267
Epoch 82/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5230 - val_loss: 0.5267
Epoch 83/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5229 - val_loss: 0.5266
Epoch 84/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5229 - val_loss: 0.5267
Epoch 85/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5228 - val_loss: 0.5266
Epoch 86/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 59us/sample - loss: 0.5340 - val_loss: 0.5283
Epoch 53/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5340 - val_loss: 0.5286
Epoch 54/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.5339 - val_loss: 0.5287
Epoch 55/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5337 - val_loss: 0.5280
Epoch 56/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5337 - val_loss: 0.5281
Epoch 57/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5336 - val_loss: 0.5277
Epoch 58/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5337 - val_loss: 0.5278
Epoch 59/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.5335 - val_loss: 0.5276
Epoch 60/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5335 - val_loss: 0.5275
Epoch 61/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 58us/sample - loss: 0.5477 - val_loss: 0.6507
Epoch 28/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5451 - val_loss: 0.6471
Epoch 29/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5428 - val_loss: 0.6439
Epoch 30/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.5406 - val_loss: 0.6407
Epoch 31/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5386 - val_loss: 0.6383
Epoch 32/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5368 - val_loss: 0.6351
Epoch 33/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.5354 - val_loss: 0.6326
Epoch 34/100
7740/7740 [==============================] - 0s 59us/sample - loss: 0.5337 - val_loss: 0.6302
Epoch 35/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.5325 - val_loss: 0.6280
Epoch 36/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 61us/sample - loss: 0.7949 - val_loss: 0.6778
Epoch 3/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.6371 - val_loss: 0.5791
Epoch 4/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.5899 - val_loss: 0.5564
Epoch 5/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.5385 - val_loss: 0.5253
Epoch 6/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.5104 - val_loss: 0.5115
Epoch 7/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4970 - val_loss: 0.4989
Epoch 8/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4895 - val_loss: 0.4902
Epoch 9/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4819 - val_loss: 0.4856
Epoch 10/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4765 - val_loss: 0.4793
Epoch 11/100
7740/7740 [==============================] -

...

7740/7740 [==============================] - 1s 67us/sample - loss: 0.4539 - val_loss: 0.4560
Epoch 18/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.4510 - val_loss: 0.4506
Epoch 19/100
7740/7740 [==============================] - 1s 68us/sample - loss: 0.4479 - val_loss: 0.4476
Epoch 20/100
7740/7740 [==============================] - 0s 64us/sample - loss: 0.4452 - val_loss: 0.4451
Epoch 21/100
7740/7740 [==============================] - 1s 66us/sample - loss: 0.4431 - val_loss: 0.4424
Epoch 22/100
7740/7740 [==============================] - 1s 67us/sample - loss: 0.4416 - val_loss: 0.4410
Epoch 23/100
7740/7740 [==============================] - 0s 64us/sample - loss: 0.4398 - val_loss: 0.4394
Epoch 24/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.4384 - val_loss: 0.4382
Epoch 25/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.4381 - val_loss: 0.4355
Epoch 26/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 62us/sample - loss: 0.3803 - val_loss: 0.3748
Epoch 94/100
7740/7740 [==============================] - 0s 64us/sample - loss: 0.3802 - val_loss: 0.3744
Epoch 95/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.3786 - val_loss: 0.3750
Epoch 96/100
7740/7740 [==============================] - 1s 68us/sample - loss: 0.3793 - val_loss: 0.3739
Epoch 97/100
7740/7740 [==============================] - 1s 70us/sample - loss: 0.3776 - val_loss: 0.3735
Epoch 98/100
7740/7740 [==============================] - 1s 67us/sample - loss: 0.3781 - val_loss: 0.3722
Epoch 99/100
7740/7740 [==============================] - 1s 65us/sample - loss: 0.3770 - val_loss: 0.3725
Epoch 100/100
3870/3870 [==============================] - 0s 33us/sample - loss: 0.3792
Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 95us/sample - loss: 1.3873 - val_loss: 0.7842
Epoch 2/100
7740/7740

7740/7740 [==============================] - 0s 64us/sample - loss: 0.3955 - val_loss: 0.3896
Epoch 69/100
7740/7740 [==============================] - 0s 64us/sample - loss: 0.3944 - val_loss: 0.3932
Epoch 70/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.3925 - val_loss: 0.3900
Epoch 71/100
7740/7740 [==============================] - 0s 64us/sample - loss: 0.3922 - val_loss: 0.3875
Epoch 72/100
7740/7740 [==============================] - 1s 66us/sample - loss: 0.3904 - val_loss: 0.3874
Epoch 73/100
7740/7740 [==============================] - 1s 70us/sample - loss: 0.3909 - val_loss: 0.3936
Epoch 74/100
7740/7740 [==============================] - 1s 66us/sample - loss: 0.3907 - val_loss: 0.4046
Epoch 75/100
7740/7740 [==============================] - 1s 68us/sample - loss: 0.3962 - val_loss: 0.3861
Epoch 76/100
7740/7740 [==============================] - 1s 66us/sample - loss: 0.3885 - val_loss: 0.3845
Epoch 77/100
7740/7740 [==========================

7740/7740 [==============================] - 0s 61us/sample - loss: 0.4016 - val_loss: 0.4115
Epoch 44/100
7740/7740 [==============================] - 0s 63us/sample - loss: 0.4007 - val_loss: 0.4090
Epoch 45/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.3991 - val_loss: 0.4082
Epoch 46/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.3980 - val_loss: 0.4088
Epoch 47/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.3972 - val_loss: 0.4055
Epoch 48/100
7740/7740 [==============================] - 0s 60us/sample - loss: 0.3961 - val_loss: 0.4063
Epoch 49/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.3949 - val_loss: 0.4047
Epoch 50/100
7740/7740 [==============================] - 0s 62us/sample - loss: 0.3939 - val_loss: 0.4035
Epoch 51/100
7740/7740 [==============================] - 0s 64us/sample - loss: 0.3929 - val_loss: 0.4024
Epoch 52/100
7740/7740 [==========================

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x14a2896a0>, as the constructor either does not set or modifies parameter learning_rate

Here we pass extra parameters to the fit() method, and they get relayed to the underlying Keras models. Note that RandomizedSearchCV uses K-fold cross validation, so it doesn't use X_valid and y_valid, which are only used for early stopping.

The exploration may last many hours, depending on the hardware, the size of the dataset, the complexity of the model, and the values of n_iter and cv. When it's over, you can access the best parameter found, the best score, and the trained Keras model like this:

In [ ]:
rnd_search_cv.best_params_

In [ ]:
rnd_search_cv.best_score_

In [ ]:
model = rnd_search_cv.best_estimator_.model

You can save this model, evaluate it on the test set, and, if you are satisfied with it's performance, deploy it to production. Using randomized search is not too hard, and it works well for many fairly simple problems.When training is slow, however(e.g. for more complex problems with larger datasets), this approach will only explore a tiny portion of the hyperparameter space. You can partially alleviate this problem by assisting the search process manually: first run a quick random search using wide range of hyperparameter values, the run another search using smaller ranges of values centered on the best one found during the first run, and so on. This approach will hopefully zoom in on a god set of hyperparameters. However, it's very time consuming, and probably it's not best use of your time.

Fortunately, there are many techniques to explore a search space much more efficiently than randomly. Their core idea is simple: when a region of the space turns out to be good, it should be explored more. Such techniques take care of the 'zooming' process for you and lead much better solutions in much less time. Here are some Python libraries you can use to optimize hyperparameters:

Hyperopt (https://github.com/hyperopt/hyperopt)
    
       
       A popular library for optimizing over all sorts of complex search spaces (including real values, such as the
       learning rate, and discrete values, such as the number of layers)
    
Hyperas (https://github.com/maxpumperla/hyperas), kopt (https://github.com/Avsecz/kopt), or Talos 
    (https://github.com/autonomio/talos)
    
       Useful libraries for optimizing hyperparameters for Keras models (the first two are based on Hyperopt)
    
Keras Tuner (https://homl.info/kerastuner)
    
       An easy-to-use hyperparameter optimization library by Google for Keras models, with a hosted service for
       visualizations and analysis.
    
Scikit-Optimize (skopt) (https://scikit-optimize.github.io/)
    
       A general purpose optimization library. The BayesSearchCV class performs Bayesian optimization using an 
       interface similar to GridSearchCV.
    
Spearmint (https://github.com/JasperSnoek/spearmint)
    
       A Bayesian optimization library.
    
Hyperband (https://github.com/zygmuntz/hyperband)
    
       A fast hyperparameter tuning library based on the recent Hyperband paper 
   (https://homl.info/hyperband) by Lisha Li et al.
    
Sklearn-Deap (https://github.com/rsteca/sklearn-deap)
    
       A hyperparameter optimization library based on evolutionary algorithms, with a GridSearchCV-like interface.

Morover, many companies offer services for hyperparameter optimization. We'll discuss Google Cloud AI Platform's hyperparameter tuning service (https://homl.info/googletuning) later. Other options include services by Arimo (https://arimo.com/) and SigOpt (https://sigopt.com/), and CallDesk's Oscar (https://oscar.calldesk.ai/).

Hyperparameter tuning is still active area of research, and evolutionary algorithms are making a comeback. 
For example, check out DeepMind's excellent 2017 paper (https:homl.info/pbt) where the authors jointly optimize a population of models and their hyperparameters. Google has also used an evalutionary approach, not just to search for hyperparameters. Google has also used an evalutionary approach, not just to search for hyperparameters but also to lookt for the best neural network architecture problem; their AutoML suite is already available as a cloud service (https://cloud.google.com/automl/). Perhaps the days of building neural networks manually will soon be over? Check out Google's post (https://homl.info/automlpost) on this topic. In fact, evolutionary algorithms have been used successfully to train individual neural networks, replacing the ubiquitous Gradient Descent! For an example, see the 2007 post (https://homl.info/neuroevol) by Uber where the authors introduce their Deep Neuroevolution technique.

But despite all this exciting process and all these tools and services, it still helps to have an idea of what values are reasonable for each hyperparameter so that you can build a quick prototype and restrict the search space. The following sections provide guidelines for choosing the number of hidden layers and neurons in an MLP and for selecting good values for some of the main hyperparameters.

### Number of Hidden Layers